# Assignment model Auction

OR-Tools version of auction algorithm

OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming.

https://github.com/google/or-tools

http://google.github.io/or-tools/

https://github.com/yu-jeremy/Auction-Algorithms/blob/master/assignmentModel.py


In [54]:
from ortools.linear_solver import pywraplp
import random

avg_across_diff_n = {}
averages = []
solver_times = []

#   n = Agents
#   M = Value bid

def main_auction(n, M):

    # Creating our mixed integer problem solver
    solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    possible_vals = []
    for i in range(0, int(M)):
        possible_vals.append(i)

    # create our value matrix, randomly assigning values from the sequence above
    value = []
    for j in range(0, int(n)):
        sublist = []
        for k in range(0, int(n)):
            val = random.choice(possible_vals)
            sublist.append(val)
        value.append(sublist)

    num_agents = len(value)
    num_objects = len(value[0])
    x = {}

    # insert decision variables into dictionary
    for i in range(num_agents):
        for j in range(num_objects):
            x[i,j] = solver.BoolVar('x[%i, %i]' % (i,j))

    # obj. function: we try to maximize value of the pairings
    solver.Maximize(solver.Sum([value[i][j] * x[i,j] for i in range(num_agents) for j in range(num_objects)]))

    # each agent is assigned to exactly one object
    for i in range(num_agents):
        solver.Add(solver.Sum([x[i, j] for j in range(num_objects)]) == 1)

    # each object is assigned to exactly one agent
    for j in range(num_objects):
        solver.Add(solver.Sum([x[i, j] for i in range(num_agents)]) == 1)

    solver.Solve()
    print('Total Value = ', solver.Objective().Value())
    print()

    value_sum = 0
    for i in range(num_agents):
        for j in range(num_objects):
            if x[i, j].solution_value() > 0:
                value_sum += value[i][j]
                print('Agent %d assigned to task %d. Value = %d' % (
                    i,
                    j,
                    value[i][j]
                ))

    average = value_sum / int(n)
    averages.append(average)
    print("Per-Agent Avg. Value of Assignments = " + str(average))
    print()
    #print("Time = ", solver.WallTime(), " milliseconds")
    solver_times.append(solver.WallTime())
    return value_sum

def auction_algorithm(bid, agent, max_bid, x_times):
    #-----------------------------------------------------------------------------
    # bid_number = number bidding
    # agent_number = number agents
    # max_value_bid = maximum value for bid
    print('------- -----------------------------')
    print('|------ Auction Algotithm --', x_times, '-----|')
    print('------- -----------------------------')   
    print('Bid Number   :', bid)
    print('Number agents:', agent)
    print('Max bid value:', max_bid)    
    
    bid_number = bid
    agent_number = agent
    max_value_bid = max_bid
    
    M = max_value_bid
    for i in range(0, bid_number):
        n = agent_number
        print("Run Bid: " + str(i+1))
        
        x_value = main_auction(n, M)
        avg_across_diff_n[n] = averages
    s = 0
    c = 0
    for key in avg_across_diff_n:
        for val in avg_across_diff_n[key]:
            s += val
            c += 1
        average = s / c
        #print("Finish Per-Agent Avg. Assignment Value: " + str(average))

    su = 0
    tt = 0
    for time in solver_times:
        su += time
        tt += 1
    time_avg = su / tt
    #print("Avg. Time for Solver to Solve Instance: " + str(time_avg))
    return x_value

In [53]:
'''
#
# This Block is only for tests
#
import numpy as np
import random
import pandas as pd

# bid_number = num. licitaçoes
# agent_number = numero agentes
# max_value_bid = maximum value for bid

bid_number = 1
agent_number = 7  # Number agents from the matrix
max_value_bid = 10
tasks = 7

proc_number = int (tasks / agent_number )
rest = tasks - (proc_number * agent_number)
print('============= auction algorith ======================')
print('Number Tasks:', tasks,'  Number Cicles:', proc_number, ' And rest tasks:',rest)

if rest > 0:
    proc_number += 1
    
# Call auction algorithm
x_times = 1
x_agent_number = agent_number
x_total = 0
while x_times <= proc_number:
    if proc_number == x_times and rest !=0 :  # last cicle
        x_agent_number = rest   # number agents needs to 
        
    rest = tasks - ((proc_number - 1 ) * agent_number)
    print('..')
    print('Auction algorithm - Cicle:', x_times, '   Number agents:', x_agent_number)
    x_total2 = auction_algorithm(bid_number, x_agent_number, max_value_bid, x_times)
    x_times += 1   
    x_total = x_total + x_total2

# registo de valores no dataframe
x_alg   = 'Auction'
x_agent = agent_number
x_task  = tasks
df = pd.DataFrame([([x_alg, x_agent, x_task, x_total ])], columns=['Algoritmo','Agents', 'Tasks', 'Cost'])
df    
'''    

"\n#\n# This Block is only for tests\n#\nimport numpy as np\nimport random\nimport pandas as pd\n\n# bid_number = num. licitaçoes\n# agent_number = numero agentes\n# max_value_bid = maximum value for bid\n\nbid_number = 1\nagent_number = 7  # Number agents from the matrix\nmax_value_bid = 10\ntasks = 7\n\nproc_number = int (tasks / agent_number )\nrest = tasks - (proc_number * agent_number)\nprint('============= auction algorith ======================')\nprint('Number Tasks:', tasks,'  Number Cicles:', proc_number, ' And rest tasks:',rest)\n\nif rest > 0:\n    proc_number += 1\n    \n# Call auction algorithm\nx_times = 1\nx_agent_number = agent_number\nx_total = 0\nwhile x_times <= proc_number:\n    if proc_number == x_times and rest !=0 :  # last cicle\n        x_agent_number = rest   # number agents needs to \n        \n    rest = tasks - ((proc_number - 1 ) * agent_number)\n    print('..')\n    print('Auction algorithm - Cicle:', x_times, '   Number agents:', x_agent_number)\n    x_